In [ ]:
!pip install SciExpeM-API 
!pip install SciExpeM-API --upgrade

In [ ]:
from SciExpeM_API.SciExpeM import SciExpeM
import os 
my_sciexpem = SciExpeM(username='nikita.rozov', password='mdp2022_')

Attention. SciExpeM is a singleton.


In [ ]:
result_list = my_sciexpem.filterDatabase(model_name='Experiment', fileDOI='tmp_39904519421883596702')

print(result_list)
for result in result_list:
    print(result.__dict__)
    print("DOI:", result.fileDOI)
    print("Reactor:", result.reactor)
    print('Data columns', result._data_columns)
    for data in result._data_columns:
      print(data.__dict__)
    

[]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
data = pd.read_excel("/content/drive/My Drive/MDP/report/POLIMI_1212_C1C3.xlsx")
for filename in os.listdir("/content/drive/My Drive/MDP/report/"):
  if filename != "POLIMI_1212_C1C3.xlsx":
    inter_data = pd.read_excel("/content/drive/My Drive/MDP/report/"+filename)
    data = pd.concat([data, inter_data], ignore_index=True)
data = data.drop(columns=["Unnamed: 0"])
print(data.shape)

(861, 18)


In [ ]:
x = data.drop(columns=['Experiment DOI','Exp SciExpeM ID','Chem Model','Error','d0L2','d1L2','d0Pe','d1Pe','shift'])
x_exp_type = pd.get_dummies(x['Experiment Type'], prefix='Experiment Type')
# dx = x.set_index('id').join(x_exp_type.set_index('image_id'))
y = data['Error']
x.head()

,Chem Model ID,Experiment Type,Reactor,Target,Fuels,Phi,Pressure (Bar),Temperature (K),Score
0,5,ignition delay measurement,shock tube,tau_P(slope),['CH4'],"(1.0, 1.0)","(2.537, 2.895)","(1405.4, 1604.1)",0.790593
1,5,ignition delay measurement,shock tube,tau_P(slope),['CH4'],"(0.999, 0.999)","(0.329, 0.329)","(1763.23, 2017.34)",0.674950
2,5,ignition delay measurement,shock tube,tau_P(slope),['CH4'],"(0.204, 0.204)","(3.04, 4.07)","(1463.0, 1705.0)",0.402664
3,5,ignition delay measurement,shock tube,tau_P(slope),['CH4'],"(1.0, 1.0)","(13.175, 13.175)","(1500.55, 1894.81)",0.915565
4,5,ignition delay measurement,shock tube,tau_P(slope),['CH4'],"(1.0, 1.0)","(2.761, 3.68)","(1593.0, 1915.0)",0.778480


In [ ]:
def splitter(x, num):
  first, second = x.replace('(','').replace(')','').split(", ")
  if num == 1:
    return first
  else:    
    return second
x['Phi1'] = x.Phi.apply(splitter,num = 1)
x['Pressure1'] = x['Pressure (Bar)'].apply(splitter,num = 1)
x['Temperature1'] = x["Temperature (K)"].apply(splitter,num = 1)
x['Phi2'] = x.Phi.apply(splitter,num = 2)
x['Pressure2'] = x['Pressure (Bar)'].apply(splitter,num = 2)
x['Temperature2'] = x["Temperature (K)"].apply(splitter,num = 2)
x = x.drop(columns=['Phi','Pressure (Bar)','Temperature (K)'])

x.head()

,Chem Model ID,Experiment Type,Reactor,Target,Fuels,Score,Phi1,Pressure1,Temperature1,Phi2,Pressure2,Temperature2
0,5,ignition delay measurement,shock tube,tau_P(slope),['CH4'],0.790593,1.0,2.537,1405.4,1.0,2.895,1604.1
1,5,ignition delay measurement,shock tube,tau_P(slope),['CH4'],0.674950,0.999,0.329,1763.23,0.999,0.329,2017.34
2,5,ignition delay measurement,shock tube,tau_P(slope),['CH4'],0.402664,0.204,3.04,1463.0,0.204,4.07,1705.0
3,5,ignition delay measurement,shock tube,tau_P(slope),['CH4'],0.915565,1.0,13.175,1500.55,1.0,13.175,1894.81
4,5,ignition delay measurement,shock tube,tau_P(slope),['CH4'],0.778480,1.0,2.761,1593.0,1.0,3.68,1915.0


In [ ]:
x_fuels = pd.get_dummies(x['Fuels'], prefix='Fuels:')
#model id to categorical
x_model_id = pd.get_dummies(x['Chem Model ID'], prefix='Model ID:')
x_reactor = pd.get_dummies(x['Reactor'], prefix='Reactor:')
x_exp_type = pd.get_dummies(x['Experiment Type'], prefix='ExpType:')
x_target = pd.get_dummies(x['Target'], prefix='Target:')
x = x.drop(columns=["Fuels",'Target','Experiment Type','Reactor','Chem Model ID'])
x.join(x_target).join(x_exp_type).join(x_reactor).join(x_fuels).join(x_model_id)

,Score,Phi1,Pressure1,Temperature1,Phi2,Pressure2,Temperature2,Target:_AR_x,Target:_BIPHENYL_x,Target:_C10H8_x,...,Model ID:_11,Model ID:_12,Model ID:_14,Model ID:_15,Model ID:_17,Model ID:_23,Model ID:_24,Model ID:_31,Model ID:_32,Model ID:_33
0,0.790593,1.0,2.537,1405.4,1.0,2.895,1604.1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.674950,0.999,0.329,1763.23,0.999,0.329,2017.34,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.402664,0.204,3.04,1463.0,0.204,4.07,1705.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.915565,1.0,13.175,1500.55,1.0,13.175,1894.81,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.778480,1.0,2.761,1593.0,1.0,3.68,1915.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
856,0.571456,0.6,20.0,423.0,0.8,20.0,423.0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
857,0.297652,2.0,2.691,1426.3,2.0,3.12,1653.6,0,0,0,...,0,0,0,0,0,0,0,1,0,0
858,0.903743,1.0,6.81,1382.42,1.0,6.81,1766.72,0,0,0,...,0,0,0,0,0,0,0,1,0,0
859,0.602642,0.2,2.883,1527.6,0.2,3.375,1788.7,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.model_selection import train_test_split
x_train, x_test = train_test_split(x, test_size=0.2)
y_train, y_test = train_test_split(y, test_size=0.2)
clfLin = LinearRegression()
clf = Ridge(alpha=1.0)
clf.fit(x_train, y_train)
clfLin.fit(x_train, y_train)

LinearRegression()

In [ ]:
predictions = clf.predict(x_test)
predictionsLin = clfLin.predict(x_test)
from sklearn import metrics
print(metrics.r2_score(y_test,predictions))
print(metrics.explained_variance_score(y_test,predictions))
print(metrics.mean_squared_error(y_test,predictions))

-2.8889191430759347
-2.35934312236347
5.770102670542828e-05


In [ ]:
print(metrics.r2_score(y_test,predictionsLin))
print(metrics.explained_variance_score(y_test,predictionsLin))
print(metrics.mean_squared_error(y_test,predictionsLin))

-2.930086361901559
-2.405898147124851
5.831183672884439e-05


In [ ]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(max_depth=1, random_state=0)
regr.fit(x_train, y_train)
predictions = regr.predict(x_test)
print(metrics.r2_score(y_test,predictions))
print(metrics.explained_variance_score(y_test,predictions))
print(metrics.mean_squared_error(y_test,predictions))

-7.602642806703939
-6.808404262397091
0.00012763991846183578
